In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt

from statsmodels.graphics import tsaplots
from statsmodels.stats import diagnostic
from statsmodels.tsa.stattools import adfuller

from itertools import product

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression, Ridge
from catboost import CatBoostRegressor, cv
from sklearn.metrics import mean_absolute_error


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-mar-2022/train.csv', index_col='row_id')
test = pd.read_csv('../input/tabular-playground-series-mar-2022/test.csv', index_col='row_id')
submission = pd.read_csv('../input/tabular-playground-series-mar-2022/sample_submission.csv')

target = 'congestion'

In [ ]:
train['ts'] = train['time'].apply(lambda x: pd.Timestamp(x))
test['ts'] = test['time'].apply(lambda x: pd.Timestamp(x))

In [ ]:
eb = train.query('direction == "EB"')
nb = train.query('direction == "NB"')
sb = train.query('direction == "SB"')
wb = train.query('direction == "WB"')
ne = train.query('direction == "NE"')
sw = train.query('direction == "SW"')
nw = train.query('direction == "NW"')
se = train.query('direction == "SE"')

eb_test = test.query('direction == "EB"')
nb_test = test.query('direction == "NB"')
sb_test = test.query('direction == "SB"')
wb_test = test.query('direction == "WB"')
ne_test = test.query('direction == "NE"')
sw_test = test.query('direction == "SW"')
nw_test = test.query('direction == "NW"')
se_test = test.query('direction == "SE"')

directions = train['direction'].unique()
data_train = {'EB': eb, 'NB': nb, 'SB': sb, 'WB': wb, 'NE': ne, 'SW': sw, 'NW': nw, 'SE': se}
data_test = {'EB': eb_test, 'NB': nb_test, 'SB': sb_test, 'WB': wb_test, 'NE': ne_test, 'SW': sw_test, 'NW': nw_test, 'SE': se_test}

In [ ]:
def direct(data, x, y, data_name='train'):
    ans = data.query(f'x == {x}').query(f'y == {y}')
    row_id = ans.index
    ans = ans.set_index('ts')
    if data_name == 'train':
        return ans['congestion']
    else:
        return ans, row_id



## One example

In [ ]:
# eb

In [ ]:
# ts = eb.query('x == 0').query('y == 1').set_index('time')



In [ ]:
def stats(ts):
    res = diagnostic.acorr_ljungbox(ts)
    p_value = res['lb_pvalue']
    print(max(p_value), min(p_value))
    print(adfuller(ts.diff(80).dropna())[1])

### EB

In [ ]:
# x_arr, y_arr = eb['x'].unique(), eb['y'].unique()

# i=1

# fig, axs = plt.subplots(nrows=8, ncols=2, figsize=(24, 8))
# for x, y in product(x_arr, y_arr):
#     plt.figure(figsize=(20, 10)) 
#     ts = direct(eb, x, y)
    
#     ts.plot(title=f'x={x}, y={y}')
    
#     tsaplots.plot_acf(ts, lags=100,)
#     tsaplots.plot_pacf(ts, lags=100,)
#     plt.show()
#     stats(ts)
#     print(f'x={x}, y={y}')


In [ ]:
!pip install isodate

In [ ]:
!pip install monthdelta

In [ ]:
import inspect
from typing import Dict, Callable, List, Any
from collections import defaultdict
from copy import deepcopy

import isodate
import pandas as pd
import numpy as np
from monthdelta import monthdelta
from sklearn.linear_model import LinearRegression
from sklearn.utils import resample


def get_timedelta_from_granularity(granularity: str):
    datetime_interval = isodate.parse_duration(granularity)
    if isinstance(datetime_interval, isodate.duration.Duration):
        years, months = datetime_interval.years, datetime_interval.months
        total_months = int(years * 12 + months)
        datetime_interval = monthdelta(months=total_months)
    return datetime_interval


class BaseEstimator:
    """
    Implements get/set parameters logic, validating estimator and other
    methods, common for all estimators. This is improved version of sklearn get/set params logic,
    that also checks all parent's class parameters in addition to self
    """

    @classmethod
    def _get_param_names(cls, deep=True) -> List[str]:
        """Get parameter names for the estimator"""

        def get_param_names_for_class(cls):
            # fetch the constructor or the original constructor before
            # deprecation wrapping if any
            init = getattr(cls.__init__, 'deprecated_original', cls.__init__)
            if init is object.__init__:
                # No explicit constructor to introspect
                return []

            # introspect the constructor arguments to find the model parameters
            # to represent
            init_signature = inspect.signature(init)
            # Consider the constructor parameters excluding 'self'
            parameters = [
                p for p in init_signature.parameters.values()
                if p.name != 'self' and p.kind != p.VAR_KEYWORD
                and p.kind != p.VAR_POSITIONAL
            ]

            # Add extra_params - params, that are not listed in signature (is_fitted e.q.)
            extra_params = getattr(cls, 'extra_params', [])

            # Extract and sort argument names excluding 'self'
            return sorted([p.name for p in parameters] + extra_params)

        # get self params
        parameters = get_param_names_for_class(cls)

        # if deep get all parents params
        if deep:
            for parent_class in cls.__bases__:
                parameters.extend(get_param_names_for_class(parent_class))

        return parameters

    def get_params(self, deep: bool = True) -> Dict[str, Any]:
        """
        Get parameters for this estimator
        Parameters
        ----------
        deep : bool, default=True
            If True, will return the parameters for this estimator and
            contained subobjects that are estimators
        Returns
        -------
        params
            Parameter names mapped to their values
        """
        out = dict()
        for key in self._get_param_names():
            try:
                value = getattr(self, key)
            except AttributeError:
                value = None
            if deep and hasattr(value, 'get_params'):
                deep_items = value.get_params().items()
                out.update((key + '__' + k, val) for k, val in deep_items)

            out[key] = value

        return out

    def set_params(self, **params):
        """
        Set the parameters of this estimator.
        The method works on simple estimators as well as on nested objects.
        The latter have parameters of the form
        ``<component>__<parameter>`` so that it's possible to update each
        component of a nested object
        Parameters
        ----------
        **params
            Estimator parameters
        Returns
        -------
        self
            Estimator instance
        """

        if not params:
            # Simple optimization to gain speed (inspect is slow)
            return self
        valid_params = self.get_params(deep=True)

        nested_params = defaultdict(dict)  # grouped by prefix
        for key, value in params.items():
            key, delim, sub_key = key.partition('__')

            if key not in valid_params:
                raise ValueError('Invalid parameter %s for predictor %s. '
                                 'Check the list of available parameters '
                                 'with `estimator.get_params().keys()`.' %
                                 (key, self))

            if delim:
                nested_params[key][sub_key] = value
            else:
                setattr(self, key, value)
                valid_params[key] = value

        for key, sub_params in nested_params.items():
            valid_params[key].set_params(**sub_params)

        return self


class TimeSeriesPredictor(BaseEstimator):
    def __init__(
            self,
            granularity: str,
            num_lags: int,
            model: Callable = LinearRegression,
            mappers: Dict[str, Callable] = {},
            **kwargs
    ):

        self.granularity = granularity
        self.num_lags = num_lags
        self.model = model(**kwargs)
        self.mappers = mappers
        self.fitted = False
        self.std = None

    def transform_into_matrix(self, ts: pd.Series) -> pd.DataFrame:
        """
        Transforms time series into lags matrix to allow
        applying supervised learning algorithms
        Parameters
        ------------
        ts
            Time series to transform
        Returns
        --------
        lags_matrix
            Dataframe with transformed values
        """

        ts_values = ts.values
        data = {}
        for i in range(self.num_lags + 1):
            data[f'lag_{self.num_lags - i}'] = np.roll(ts_values, -i)

        lags_matrix = pd.DataFrame(data)[:-self.num_lags]
        lags_matrix.index = ts.index[self.num_lags:]

        return lags_matrix

    def enrich(
            self,
            lags_matrix: pd.DataFrame
    ) -> pd.DataFrame:
        """
        Adds external features to time series
        Parameters
        ------------
        lags_matrix
            Pandas dataframe with transformed time-series values
        mappers
            Dictionary of functions to map each timestamp of lags matrix.
            Each function should take timestamp as the only positional parameter
            and return value of your additional feature for that timestamp
        """

        mappers = self.mappers
        for name, mapper in mappers.items():
            feature = pd.Series(lags_matrix.index.map(mapper), lags_matrix.index, name=name)
            lags_matrix[name] = feature

        return lags_matrix

    def fit(self, ts: pd.Series, *args, **kwargs):
        lag_matrix = self.transform_into_matrix(ts)
        feature_matrix = self.enrich(lag_matrix)

        X, y = feature_matrix.drop('lag_0', axis=1), feature_matrix['lag_0']
        self.model.fit(X, y, *args, **kwargs)
        self.fitted = True

    def predict_next(self, ts_lags, n_steps=1):
        if not self.model:
            raise ValueError('Model is not fitted yet')

        predict = {}

        ts = deepcopy(ts_lags)
        for _ in range(n_steps):
            next_row = self.generate_next_row(ts)
            next_timestamp = next_row.index[-1]
            value = self.model.predict(next_row)[0]
            predict[next_timestamp] = value
            ts[next_timestamp] = value
        return pd.Series(predict)

    def predict_batch(self, ts: pd.Series, ts_batch: pd.Series = pd.Series()):
        if not self.model:
            raise ValueError('Model is not fitted yet')

        unite_ts = ts.append(ts_batch)
        matrix = self.enrich(self.transform_into_matrix(unite_ts))

        data_batch = matrix[-len(ts_batch):]
        preds = self.model.predict(data_batch.drop('lag_0', axis=1))

        return pd.Series(index=data_batch.index, data=preds)

    def generate_next_row(self, ts):
        """
        Takes time-series as an input and returns next row, that is fed to the fitted model,
        when predicting next value.
        Parameters
        ----------
        ts : pd.Series(values, timestamps)
            Time-series to detect on
        Returns
        ---------
        feature_matrix : pd.DataFrame
            Pandas dataframe, which contains feature lags of
            shape(1, num_lags+len(external_feautres))
        """

        delta = get_timedelta_from_granularity(self.granularity)
        next_timestamp = pd.to_datetime(ts.index[-1]) + delta
        lag_dict = {'lag_{}'.format(i): [ts[-i]] for i in range(1, self.num_lags + 1)}
        df = pd.DataFrame.from_dict(lag_dict)
        df.index = [next_timestamp]
        df = self.enrich(df)

        return df

In [ ]:
import math

from pandas._libs.tslibs.timestamps import Timestamp


def get_month_sin(timestamp: Timestamp) -> float:
    theta = timestamp.month * (2*math.pi / 12)
    return math.sin(theta)


def get_month_cos(timestamp: Timestamp) -> float:
    theta = timestamp.month * (2*math.pi / 12)
    return math.cos(theta)


def get_day_sin(timestamp: Timestamp) -> float:
    theta = timestamp.day * (2*math.pi / timestamp.days_in_month)
    return math.sin(theta)


def get_day_cos(timestamp: Timestamp) -> float:
    theta = timestamp.day * (2*math.pi / timestamp.days_in_month)
    return math.cos(theta)


def get_dayofweek_sin(timestamp: Timestamp) -> float:
    theta = timestamp.dayofweek * (2*math.pi / 7)
    return math.sin(theta)


def get_dayofweek_cos(timestamp: Timestamp) -> float:
    theta = timestamp.dayofweek * (2*math.pi / 7)
    return math.cos(theta)


def get_hour_sin(timestamp: Timestamp) -> float:
    theta = timestamp.hour * (2*math.pi / 24)
    return math.sin(theta)


def get_hour_cos(timestamp: Timestamp) -> float:
    theta = timestamp.hour * (2*math.pi / 24)
    return math.cos(theta)


def get_minute_sin(timestamp: Timestamp) -> float:
    theta = timestamp.minute * (2*math.pi / 60)
    return math.sin(theta)


def get_minute_cos(timestamp: Timestamp) -> float:
    theta = timestamp.minute * (2*math.pi / 60)
    return math.cos(theta)

def get_day_name(timestamp: Timestamp) -> float:
    return timestamp.isoweekday()


datetime_mappers = {
    'month_sin': get_month_sin,
    'month_cos': get_month_cos,
    'day_sin': get_day_sin,
    'day_cos': get_day_cos,
    'dayofweek_sin': get_dayofweek_sin,
    'dayofweek_cos': get_dayofweek_cos,
    'hour_sin': get_hour_sin,
    'hour_cos': get_hour_cos,
    'minute_sin': get_minute_sin,
    'minute_cos': get_minute_cos,
    'day_name': get_day_name
}

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.signal import periodogram, detrend

def get_season_period(ts):
    ts = pd.Series(detrend(ts), ts.index)
    f, Pxx = periodogram(ts)
    Pxx = list(map(lambda x: x.real, Pxx))
    ziped = list(zip(f, Pxx))
    ziped.sort(key=lambda x: x[1])
    highest_freqs = [x[0] for x in ziped[-100:]]
    season_periods = [round(1/(x+0.001)) for x in highest_freqs]
    for period in reversed(season_periods):
        if 4 < period < 100:
            return int(period)

In [ ]:
# ts_train, ts_val = ts[:int(ts.shape[0] * 0.8)], ts[int(ts.shape[0] * 0.8):]

In [ ]:
import optuna
from optuna.samplers import TPESampler
import random
sampler = TPESampler(seed=13)

In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
def hyperparams(ts):
    
#     ts_train, ts_val = ts[:int(ts.shape[0] * 0.8)], ts[int(ts.shape[0] * 0.8):]

    def objective(trial):
        params = {'iterations':trial.suggest_int("iterations", 500, 1500),
              'od_wait':trial.suggest_int('od_wait', 100, 400),
              'task_type':"GPU",
              'eval_metric':'MAE',
              'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 0.9),
              "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
              'subsample': trial.suggest_uniform('subsample',0.9,1.0),
              'random_strength': trial.suggest_uniform('random_strength',10,50),
              'depth': trial.suggest_int('depth',2,8),
              'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
              'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
            'verbose': 500, 
                  'bootstrap_type':'Poisson'
               }
        
        res = []
        tscv = TimeSeriesSplit(n_splits=3)
        for train_index, test_index in tscv.split(ts):
            ts_train, ts_val = ts[train_index], ts[test_index]

            Model = CatBoostRegressor
        
        
            granularity = 'PT20M'
            num_lags = get_season_period(ts_train)

            predictor = TimeSeriesPredictor(
                granularity=granularity,
                num_lags=num_lags,
                model=Model,
                mappers=datetime_mappers, 
                **params
            )
        
            lags_matrix = predictor.transform_into_matrix(ts_train)
            feature_matrix = predictor.enrich(lags_matrix)
            predictor.fit(ts_train)
            preds = predictor.predict_batch(ts_train, ts_val)
            res.append(preds)
        
        res = np.mean(res, axis=0)

        return mean_absolute_error(ts_val, res)


    study = optuna.create_study(direction="minimize", sampler=sampler)
    study.optimize(objective, n_trials=2)
    params_cat = study.best_params
    return params_cat

In [ ]:
# params_cat = hyperparams(ts)

In [ ]:
# params_cat = {}
# params_cat["verbose"] = 0
# params_cat

In [ ]:
# datetime_mappers

In [ ]:

# Model = CatBoostRegressor
# params_cat = {"verbose": 0, 
#              "learning_rate": 0.9}


# granularity = 'PT20M'
# num_lags = get_season_period(ts_train)

# predictor = TimeSeriesPredictor(
#     granularity=granularity,
#     num_lags=num_lags,
#     model=Model,
#     mappers=datetime_mappers, 
#     **params_cat
    
# )

# lags_matrix = predictor.transform_into_matrix(ts_train)
# lags_matrix

In [ ]:
# feature_matrix = predictor.enrich(lags_matrix)
# predictor.mappers

In [ ]:
# feature_matrix

In [ ]:
# predictor.fit(ts_train)
# preds = predictor.predict_next(ts_train, n_steps=len(ts_val))

In [ ]:
# preds.plot()
# ts_val.plot()

# mean_absolute_error(ts_val, preds)

In [ ]:
# eb_test = test.query('direction == "EB"')

# eb_test, row_id = direct(eb_test, 2, 3, 'test')

# eb_test['congestion'] = 0
# ts_test = eb_test['congestion']
# eb_test

In [ ]:
# prediction = predictor.predict_next(ts, n_steps=len(eb_test))

In [ ]:
# ts[-100:].plot()
# prediction.plot()

In [ ]:
# row_id

# ALL

In [ ]:
tmp_df = submission.set_index('row_id').copy()


# Model = GradientBoostingRegressor
Model = CatBoostRegressor
granularity = 'PT20M'

In [ ]:
for a, b in data_train.items():
    test_tmp = data_test[a]
    x_arr, y_arr = b['x'].unique(), b['y'].unique()

    for x, y in product(x_arr, y_arr):
        print(f'Calc name={a}, direction={x, y}')
        ################# collect ts #############
        ts = direct(b, x, y)

        if len(ts) != 0:
            stats(ts)
            ts_test, row_id = direct(test_tmp, x, y, 'test')
            ################## tune params #############
            params_cat = hyperparams(ts)
            params_cat["verbose"] = 0
#             params_cat = {"verbose": 0}
            ################# fit model #############
            
            num_lags = get_season_period(ts)

            predictor = TimeSeriesPredictor(
                                            granularity=granularity,
                                            num_lags=num_lags,
                                            model=Model,
                                            mappers=datetime_mappers,
                                             **params_cat
                                        )

            lags_matrix = predictor.transform_into_matrix(ts)

            predictor.fit(ts)
            prediction = predictor.predict_next(ts, n_steps=len(ts_test))

            d = {'row_id': row_id, 'congestion': prediction}
            res = pd.DataFrame(data=d)

            tmp_df.loc[row_id, 'congestion'] = prediction.values
            print('='*50)
        else:
            print(f'!!Warning!! name={a}, direction={x, y} not found')
            print('='*50)


In [ ]:

tmp_df.reset_index().to_csv('./submission_1model.csv', index=False)